# Documentation -- Milestone 2 

## Introduction

Differentiation is great. It is a necessity in a vast range of applications, such as atomic simulations, economic analysis, and machine learning. 

In the field of computational science, there are three ways to diffrentiate: numerical, sybolic and automatic. 

Numerical differentiation finds the derivative using finite difference approximations $\Delta f / \Delta x$. Even with higher-order methods, its error is far greater than machine precision.

Symbolic differentiation finds the symbolic expression of the derivative. When functions and programs get complicated, it becomes inefficient and messy. This is called expression swell.

Automatic differentiation can find the derivative of expressions to the accuracy of machine precision. It does not have the problem of expression swell because it deals with numbers. That is why we need automatic differntiation!

Our `superdiff` package performs automatic differentiation on single- or multi-variable functions using the _forward mode_ as well as the _reverse mode_. The function is stored as an `Expression` object that can output values and derivatives at any given point.


## Background

Differentiation is the process of finding derivative, which is the rate of change of a function's output with regard to its variables. Take $f(x,y) =3*x^2+\exp(y)$ as an example. Symbolic differentiation gives $\dfrac{\partial f}{\partial x}=6x$ and $\dfrac{\partial f}{\partial y}=\exp(y)$.

Automatic differentiation treats a function as a chain of elementary functions and performs differentiation on each elementry function. 
Here the elementary functions include: (1) A single arithmetic operation, such as $3*x$ and $x_1+x_2$. (2) A single trigonometric operation, such as $\sin(x)$. (3) A single exponential or logarithmic opration, such as $\log(x)$.

The chain rule dictates that 

$$\frac{df(g(x))}{dx}=\frac{df(x_1)}{dx_1}*\frac{dg(x)}{dx}.$$

Therefore, a function that is made up of elementary functions can be extended into a computational graph. For $f(x,y) =3*x^2+\exp(y)$, the graph is shown below. Each $x_i$ is the output of an elementary function.

<img src="https://i.imgur.com/hBQvv4n.jpg" alt="drawing" width="600"/>
  
To calulate the derivative of $f$ at $[x,\ y]$, we pass the value of the previous $x_i$ and $x_i^\prime$ into the next elementary function to evaluate the derivative of that elementary function. Below shows the forward mode automatic differentiation table (traceplot). 

<img src="https://i.imgur.com/1AIngxT.png" alt="drawing" width="600"/>

The derivative is computed using the chain rule. To get $\dfrac{\partial f}{\partial x}$, forward mode starts from $\dfrac{\partial x_1}{\partial x}$, while the reverse mode starts from $\dfrac{\partial x_6}{\partial x_4}$. 

Let's do the forward mode at a given point x=2, y=3. Start from the begining, plug in numbers for each step, we have

$$\dfrac{\partial f}{\partial x} = \dfrac{\partial x_6}{\partial x_4}\dfrac{\partial x_4}{\partial x_3}\dfrac{\partial x_3}{\partial x_1}\dfrac{\partial x_1}{\partial x}=1*3*4*1=12.$$

## How to use superdiff

Our goal is for the syntax of `superdiff` to be as natural as possible, not requiring the user to learn any new paradigms and thereby minimizing the chances of hard-to-debug errors. Therefore, we take inspiration from the kind of notation one might use when writing out mathematical expressions and functions by hand. 

The core functionality of `superdiff` involves three main kinds of objects: `Variable`, `Expression`, and subclasses of `Operation`. These mean exactly what you might expect from a mathematical context. If a user wants to define an expression, they first define one or more `Variable`s. The they make an `Expression` using basic math operators such as `+ - * / ` or special operators such as `superdiff.log`. The expression can be evaluated and differntiated at any given point.


### 1. How to install `superdiff`
```python
pip install
```

### 2. Demo
```python
import superdiff as sd

# Define the base variable
x = sd.Var('x')

# A variable behaves like the identity operation
x(-24)
>>> -24

# And has a derivative of 1
x.deriv(-24)
>>> 1

# The power of superdiff lies in the ability to compose arbitrarily complex functions, called Expressions.
# This creates an `Expression` with the specified operations
f = sd.make_expression((x * 0.2 + sd.exp(x) / 3) / x, vars = [x])

# We can evaluate it just like a function
f(2)
>>> 1.431509349821775

# As well as take the derivative using forward mode
f.deriv(2)
>>> 0.6157546749108876

# To define more complex functions, we can combine previously-made expressions
g = sd.make_expression(f / 4 + x ** 2, vars=[x])

# Evaluating and taking derivatives:
g(2)
>>> 4.357877337455443

g(2) == f(2) / 4 + 2 ** 2
>>> True

g.deriv(2)
>>> 4.153938668727722

g.deriv(2) == f.deriv(2) / 4 + 2 * 2
>>> True

```

## Software Organization
### 1. Directory structure
```
cs207-FinalProject/
|
|-- superdiff/
|   |-- __init__.py
|   |-- superdiff.py
|   |-- expression.py
|   |-- operations.py
|
|-- tests/
|   |-- conftest.py
|   |-- operations_test.py
|   |-- test_dummy.py
|   |-- test_expression.py
|
|-- docs/
|   |-- milestone1.md
|   |-- milestone2.ipynb
|
|-- README.md
|-- requirements.txt
|-- setup.py
|-- LICENSE
```

`superdiff/` subdirectory hosts our code. It is set up as a package through `__init__.py`.

`tests/` subdirectory hosts tests to the code.

`docs/` subdirectory hosts our milestone documents. The documents provides an introduction to automatic differntiation, as well as a guide to using our package.

### 2. Basic modules and their functionality

Our modules are `__init__.py`, `superdiff.py`, `expression.py`, and `operations.py`. 

1. `__init__.py` treats `superdiff` as a package.

2. `superdiff.py` imports everthing from `expression.py` and `operations.py`. This contains the user's primary interface through the `make_expression` function, which abstracts away the graph structure of autodiff and easily allow the user to create an expression. It also contains a set of basic math operations the user can utilize in their expressions, like $sin(x)$ or $exp(x)$. 

3. `expression.py` contains the code for our `Variable` and `Expression` classes. These objects build the computational graph in a tree-like structure. Users can call the `eval()` and `deriv()` methods of an expression to get values and derivatives.

4. `operations.py` contains elementary function classes. Each elementary function have methods to build new expressions, to evaluate at given points, and to compute the derivative at given points.

### 3. Where do tests live? How are they run? How are they integrated?

As shown in the directory structure, our test suite lives inside the `tests/` subdirectory. 

The primary tests lies in `operations_test.py` and `test_expression.py`. Both include many unit tests to ensure the differentiator modules run correctly and handle edge cases, including type checking, appropriately for a variety of complex functions. Both also ensure that basic math operators are properly overloaded when users create expressions and evaluate them at certain points.

- `operations_test.py` more closely tests many of the math operations abstracted away under the hood to ensure quality is maintained beneath all the layers.

- `test_expression.py` more closely ensures that `Expression` objects are instantiated properly and return correct evaluations and deriatives. 

`pytest` runs the test suite. TravisCI performed integration testing as we built the package to ensure proper functionality among all the moving pieces in our code, and it helped flag defects as they arose and maintained quality control among the various components in the software. In addition, CodeCov helped us analyze ways to improve our test suite to maintain high coverage of our code.

### 4. How to install our package

As of now, users may install our package by downloading the project repository and running `pip install /path/to/package`. They may also run `python setup.py install` within the project repository. 

In the next step, we will use PyPI to distribute our package and use `pip` to package the software. In `setup.py`, we include a brief description of the package, authors and licensing information, and the appropriate version of the package so that we can host our package on PyPI. This will allow potential users to easily install our product through the `pip install` command.


## Implementation
### 1. Core data structures

The function to be differentiated (henceforth referred to as ƒ) will be parsed and converted into a directed graph, containing node-like objects for each step in the traceplot (i.e. each node represents an elementary operation in ƒ). The edges of the graph represent steps from one part of the traceplot to the next. This is similar to the tree structure that we learned in class, but it is built from the leaf nodes to the root.

Each node is an `Expression` object that contains
- The type of elementary operation being performed. It could be add, mul, pow, log, etc. 
- References to the mathematical objects ('parents') that go into this operation. A binary operation has two parents, while a unary operation has only one parent. The parent could be an `Expression` object, a `Variable` object, or just a number.
- Notice that a node does not reference the next operation to be performed. 

Say we have the following situation:

![](https://i.imgur.com/p2gMe9B.png)


In this case, C knows about A and B, but not about F. This may seem counterintuitive, since in the forward mode of autodiff, we need to go from A to C to F. However, we want to allow for situations like the following:

![](https://i.imgur.com/eWljQhb.png)


Here, the function `f3` is composed of two inputs (`f1` and `f2`) and combines them in an operation in node G. Rather than copying functions `f1` and `f2` into brand new graphs, we believe that it would be more memory- and time-efficient to simply refer to the same graph objects that `f1` and `f2` refer to. However, this creates a potential issue if we were to implement the graphs as bidirectional, rather than unidirectional: if we add a connection from C to G, then if the user tries to run the forward mode of autodiff on `f1`, the algorithm will continue past node C onto node G. However, node G is not part of function `f1`!

This design choice has a tradeoff, namely that each time forward mode auto differentiation is performed, Python must step from the end of the function all the way to the beginning leaf nodes. This is done by recursively calling the `eval()` and `deriv()` methods.

### 2. Core classes

<img src="https://i.imgur.com/ST3mu2D.png" alt="drawing" width="800"/>


`Var` class is the variable in the eventual function. It overloads math operators such as `__add__` and `__pow__`.


`Expression` class inherits from `Var`. It stores the user-defined functions. As shown in the demo, `f` and `g` are expressions.


`BaseOperation` and its subclasses. Each operation has class methods to return new expression objects, to return the numerical value, and to return the drivative.


### 3. Important attributes

`Var` class

- `eval()` method. Var object do not store values. It returns whatever value that is being passe into it.
- `deriv()` method. If we are differentiating with respect to this particular variable, it returns 1. If we are differentiating with respect to another variable, it returns 0. This is done by passing a boolean as the input.
- Math operators such as `__add__()`. It calls `superdiff.add`, which returns an Expression object.


`Expression` class

- All methods defined in `Variable`
- `self.parents`. The parents can be an Expression object, a Var object, or a number.
- `self.operation` stores the operation that is being performed.
- `self._vars` stores the list of variables but do not store their values. 
- `eval()` and `deriv()` use helper methods to recursively find the value and derivative at any given point. Currently `deriv` only works with the forward mode.
- `_parse_args()` and `_get_input_args()` are helper methods that take in numerical input and smartly passes the values to the parent expressions. They can handle the situation where two parent expressions use different variables, e.g. `f(x) * g(y)`.


Operation classes such as `Add` and `Log`

- `expr()` takes in parent nodes and returns a new Expression object.
- `eval()` takes in numbers to evaluate this operation at any given point.
- `deriv()` takes in numbers to find the derivative at any given point. For example, `Mul.deriv()` returns `num1 * deriv2 + num2 * deriv1`.

`make_expression` function 

- It returns an expression with the varlist in the specified order. In this way, a user can control the order of the variable list.

```python
x = sd.Var('x')
y = sd.Var('y')
f1 = sd.make_expression(sd.log(x)/ y, vars = [x, y])
f2 = sd.make_expression(sd.log(x)/ y, vars = [y, x])

```


### 4. External dependencies

* `numpy`
* `pytest` (only for the test suite, not for the actual functionality)

### 5. Elementary functions

The elementary functions are coded in module `operations.py`. As described above, each operation has its own class. Currently, we have 11 operations that is working. They are then loaded into `superdiff.py` as functions. Here is how to use them:

- Simple math operations can be invoked by `+`, `-`, `*`, `/`,and `**`.
- `sd.exp(x)` exponetial function that takes in one argument.
- `sd.log(x, base)` logarithmic function that takes in two arguments. The second argument must be a number. 
- `sd.nlog(x)` natural logarithmic function that takes in one argument.
- `sd.sin(x)` sine function that takes in one argument.
- `sd.cos(x)` cosine function that takes in one argument.
- `sd.tan(x)` tangent function that takes in one argument.

## Future plan

 Next we will implement automatic differentiation on multi-variable functions and vectorized variables. We will also implement the reverse mode.
 - Multi-variable functions: \
 The basic structure of handling multi-variable functions were already finished by this milestone. There should just be more tests for `Expression` class in the tests suite. 
 
 - Reverse mode:\
 We will implement reverse mode by extending functionalities in `deriv`,`_unary_deriv` and `_binary_deriv`methods in `expression.py` in order to do automatic differentiation in reverse mode. The primary challenge will be to coordinate the functions in `operation.py` with methods in `expression.py`.
    
 - Handling vectorized inputs: \
 We will extend the elementary function classes coded in module `operation.py` so that they can deal with list and array. The main challenge would be implementing the `eval` and `deriv` methods, where both methods would take lists of numbers to be evaluated at as the parameters. There will be assertions to check if the provided lists have the same length as the variables, and then do the math accordingly. 